In [715]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb

from sklearn.model_selection import train_test_split

In [716]:
import pandas as pd

In [736]:
dataset = pd.read_csv('../../dataset/baseball_train_final2.csv')

In [743]:
dataset.columns

Index(['PCODE', 'NAME', 'year', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타',
       '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '투구',
       'barrel', 'start_date', 'end_date', 'gap', '게임수', '출루', '장타',
       'series_index'],
      dtype='object')

In [738]:
dataset = dataset[dataset['타수'] >= 30]
dataset = dataset[(dataset["출루"] != 0)]

In [768]:
X_feature = ['선발', '타수', '득점', '안타', '2타', '3타', '홈런',
            '루타', '타점','볼넷', '삼진', '희타',
            '병살','barrel']
            #'LG', 'KIA', 'KT', '키움', '두산', '한화', 'NC', '롯데', '삼성', 
            #'SSG', '홈경기수', '원정경기수', ]

y_feature = '출루'

In [769]:
X = dataset[X_feature]
y = dataset.loc[:, y_feature]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [770]:
train_ds = lgb.Dataset(X_train, label = y_train) 
test_ds = lgb.Dataset(X_val, label = y_val)

In [771]:
params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

In [772]:
model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000145 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 258
[LightGBM] [Info] Number of data points in the train set: 2017, number of used features: 14
[LightGBM] [Info] Start training from score 0.429695
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

In [773]:
predict_test = model.predict(X_val)

In [774]:
mean_absolute_error(predict_test, y_val)

0.07171601477745067